In [ ]:
using FileIO
using PyPlot
using LaTeXStrings
using MixedLayerModel

┌ Info: Precompiling FileIO [5789e2e9-d7fb-5bc7-8068-2c6fae9b9549]
└ @ Base loading.jl:1278
┌ Info: Precompiling PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee]
└ @ Base loading.jl:1278
┌ Info: Precompiling MixedLayerModel [45e51734-b38e-40e9-bdad-5745a9e5fe6d]
└ @ Base loading.jl:1278


In [ ]:
out400 = load("../experiments/output/co2_400.jld2")
out500 = load("../experiments/output/co2_upstep_500.jld2")
out600 = load("../experiments/output/co2_upstep_600.jld2")
out700 = load("../experiments/output/co2_upstep_700.jld2")
out800 = load("../experiments/output/co2_upstep_800.jld2")

p = basic_params();

In [ ]:
function profile(u, maxz)
    zi, hM, qM, SST = u;
    
    stepz = 1.0;
    z1 = collect(0:stepz:zi); z2 = collect(zi:stepz:maxz);
    z = [z1;z2];
    
    h = [hM .* ones(length(z1)); h_ft(z2, p)];
    qt = [qM .* ones(length(z1)); q_ft(z2, p)];
    
    θ = theta(z,h,qt); # K
    
    T = temp.(z,h,qt);
    ql = q_l(z,T,qt) * 1000.0; # kg/kg -> g/kg
    
    return z, θ, ql 
end

In [ ]:
rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams");
rcParams["font.size"] = 15;

fig1 = figure(figsize=(10,8));
ax1 = subplot(121);
ax2 = subplot(122);

exps = [out400, out500, out600, out700, out800];
co2 = [400, 500, 600, 700, 800];
maxz = 2000;

for (i,o) in enumerate(exps)
    u = o["uf"];
    z, θ, ql = profile(u, maxz)
    ax1.plot(θ, z)
    ax2.plot(ql, z, label=string(co2[i])*" ppm")
end

ax1.set_ylim([0,maxz]); ax2.set_ylim([0,maxz]);
ax1.set_ylabel("z (m)");
ax1.set_xlabel("θ (K)"); ax2.set_xlabel("ql (g/kg)");
legend();
tight_layout();
show()

In [ ]:
rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams");
rcParams["font.size"] = 15;

fig1 = figure(figsize=(8,8));
ax1 = subplot(121);
ax2 = subplot(122);

exps = [out400, out500, out600, out700, out800];
co2 = [400, 500, 600, 700, 800];

for (i,o) in enumerate(exps)
    u = o["uf"];
    co2i = co2[i];
    zi, hM, qM, SST = u;
    lwp = calc_LWP(zi, hM, qM) * 1000;
    ax1.plot(co2i, lwp, "o")
    ax2.plot(co2i, SST, "o")
end

ax2.set_xlabel(L"CO$_2$ (ppm)"); 
ax1.set_ylabel(L"LWP (g/m$^2$)"); ax2.set_ylabel("SST (K)");
tight_layout();
show()